In [ ]:
from sklearn.datasets import make_blobs
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
import numpy as np
from scipy.stats import mode
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN

In [ ]:
# Generate synthetic data with 4 clusters
data, labels = make_blobs(n_samples=1000, centers=4, cluster_std=1.0, random_state=42)

# Convert to DataFrame
df = pd.DataFrame(data, columns=['Feature_1', 'Feature_2'])
df['True_Label'] = labels

# Visualize the generated data
plt.scatter(df['Feature_1'], df['Feature_2'], c=df['True_Label'], cmap='viridis', s=50)
plt.title("Generated Data with True Labels")
plt.xlabel("Feature_1")
plt.ylabel("Feature_2")
plt.show()